# 지난 시간 복습

1. Variable
2. Function
3. Numerical Differential

In [157]:
import numpy as np

In [158]:
class Variable(object):
    def __init__(self, data):
        self.data = data

class Function(object):
    def __call__(self, var):
        y = self.forward(var)
        return Variable(y)
    
    def forward(self):
        raise NotImplementedError
    
def numerical_diff(f, x, eps=1e-4):
    x = x.data
    return (f(Variable(x + eps)).data - f(Variable(x - eps)).data) / (2 * eps)



Let $f(x) = 10x - 5$  
then, $f'(x) = 10$  
  
Now, I'll check that numerical differential can approximate the real differential.

In [159]:
class f(Function):
    def forward(self, data):
        return 10 * data.data - 5
    
def approx(data):
    return round(data)
    
real_diff = 10
data = Variable(np.array(10))
num_diff = numerical_diff(f(), data)
print(num_diff)
print(approx(num_diff) == real_diff)

10.000000000047748
True


이번 주에는 총 3가지를 배운다.  

1. Backpropagation-Theory
2. Manual Backpropagation
3. Automatic Backpropagation
  
--- 
개인적인 공부 
1. Python magic method
2. Python decorator

# Not clone coding

책 내용을 1번만 읽고, 기억나는 기능들을 구현  

In [162]:
class Variable(object):
    def __init__(self, data: Variable):
        self.data: Variable = data
        self.grad: Variable = None
        self.parent: Function = None

    def set_parent(self, parent: Function):
        self.parent = parent

    def backward(self):
        f = self.parent
        if f is not None:
            dx = f.input
            dy = f.backward()
            dx.grad = dy
            dx.backward()

class Function(object):
    def __call__(self, var):
        self.input: Variable = var
        x = var.data

        y = self.forward(x)
        y = Variable(y)
        self.output = y

        y.set_parent(self)
        return y
    
    def forward(self):
        return NotImplementedError
    
    def backward(self):
        return NotImplementedError
    
class Exp(Function):
    def forward(self, data):
        return np.exp(data)
    
    def backward(self):
        return Variable(np.exp(self.output.grad.data))
    
class Square(Function):
    def forward(self, data):
        return data ** 2
    
    def backward(self):
        return Variable(2 * self.output.grad.data)

In [163]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

y.grad = Variable(np.array(1.0))
y.backward()
print(x.grad.data)

14.7781121978613


# 2. Manual Backpropagation

In [ ]:
class Variable(object):
    def __init__(self, data):
        self.data = data
        self.grad = None

class Function(object):
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        self.input = input
        return output